# Demonstration of control of SBIG CCD cameras

In [1]:
# Load the PANOPTES module dir
import sys
sys.path.append('../../')

In [2]:
import astropy.units as u
import os
import glob
import time

In [3]:
from pocs.camera.sbig import Camera
from pocs.camera.sbigudrv import SBIGDriver
from pocs.focuser.simulator import Focuser
from pocs.scheduler.field import Field
from pocs.scheduler.observation import Observation

In [4]:
c0 = Camera(set_point=0, port='83F011167', focuser='birger', focus_port='/dev/tty.USA49WG2P4.4')

2017-01-18 11:55:32 aaomc48as.aao.gov.au panoptes[95168] INFO 			 SBIGDriver initialised: found 1 cameras
2017-01-18 11:55:46 aaomc48as.aao.gov.au panoptes[95168] INFO 			 Birger Focuser (10858) on /dev/tty.USA49WG2P4.4 initialised
2017-01-18 11:55:47 aaomc48as.aao.gov.au panoptes[95168] INFO 			 SBIG Camera (83F011167) initialised


In [5]:
c0

In [6]:
c0.focuser

In [7]:
c0.focuser.camera

In [ ]:
c1 = Camera(set_point=0, focuser=c0.focuser, focus_port='not_needed')

In [8]:
c0.uid

'83F011167'

In [9]:
c0._info

{'AO8': False,
 'Biorad_TDI_support': False,
 'STXL': False,
 'Truesense': False,
 'bad_columns': [],
 'camera_name': 'SBIG STF-8300 CCD Camera',
 'camera_type': 'STF_CAMERA',
 'colour': False,
 'dump_extra': 0,
 'electronic_shutter': False,
 'firmware_version': '2.45',
 'frame_buffer': True,
 'frame_transfer': False,
 'imaging_ABG': True,
 'mechanical_shutter': True,
 'readout_modes': {'RM_1X1': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_1X1_VOFFCHIP': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_2X2': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 1266.0 pix>,
   'pixel_height': <Quantity 10.8 um>,
   'pixel_width': <Quantity 10.8 um>,
   'width': <Quantity 1676.0 pix>},
 

In [10]:
c0.CCD_cooling_enabled

True

In [11]:
c0.CCD_set_point

<Quantity 0.18901872786513252 deg_C>

In [12]:
c0.CCD_temp

<Quantity -0.6518153424969313 deg_C>

In [13]:
c0.CCD_cooling_power

43.92156862745098

Delete any FITS files from previous runs of this notebook

In [22]:
for fits_name in glob.glob('*.fits'):
    os.unlink(fits_name)

Basic exposure. By default this is non-blocking so sleep for a while to allow it to complete.

In [15]:
c0.take_exposure(1.0, filename='test0.fits')
time.sleep(5)

Basic dark exposure, shutter will be kept closed throughout.

In [16]:
c0.take_exposure(1.0, filename='test1.fits', dark = True)
time.sleep(5)

Take a series of dark exposures one after another using the blocking option

In [17]:
for i in range(2,5):
    c0.take_exposure(0, filename='bias{}.fits'.format(i), dark=True, blocking=True)

Start two dark exposures without waiting for the first to finish. This will generate warnings, but will still work. Don't do this though!

In [18]:
c0.take_exposure(1.0, filename='test5.fits', dark = True)
c0.take_exposure(1.0, filename='test6.fits', dark = True)
time.sleep(10)

2017-01-18 11:56:32 aaomc48as.aao.gov.au panoptes[95168] WARNING Attempt to start exposure on 0 while camera busy!
2017-01-18 11:56:32 aaomc48as.aao.gov.au panoptes[95168] WARNING Waiting for exposure on 0 to complete
2017-01-18 11:56:33 aaomc48as.aao.gov.au panoptes[95168] WARNING Waiting for exposure on 0 to complete


Take an exposure using the higher level `take_observation()` interface. This will also call `process_observation()` on the resulting FITS file.

In [19]:
field = Field('Test Observation', '20h00m43.7135s +22d42m39.0645s')
observation = Observation(field, exp_time=1.5 * u.second)
e0 = c0.take_observation(observation, headers={})

## Focusing test

In [74]:
c0.take_exposure(0.1, filename='office.fits', blocking=True)

In [72]:
c0.focuser.move_to(14360)

14363

In [1]:
for fits_name in glob.glob('*.fits'):
    os.unlink(fits_name)

NameError: name 'glob' is not defined